In [4]:
import boto3
import math
import os
import time


def s3_get_meta_data(conn, bucket, key):
    meta_data = conn.head_object(
    Bucket=bucket,
    Key=key
    )
    return meta_data

In [7]:
def s3_download(conn, bucket, key, parallel_threads):
    start = time.time()
    md = s3_get_meta_data(conn, bucket, key)
    chunk = get_cunks(md["ContentLength"], parallel_threads)
    print("Making %s parallel s3 calls with a chunk size of %s each..." % (parallel_threads, convert_size(chunk)))
    cur_dir = os.path.dirname(os.path.realpath(__file__))
    conn.download_file(
        Bucket=bucket,
        Filename=key.split("/")[-1],
        Key=key,
        Config=boto3.s3.transfer.TransferConfig(max_concurrency=parallel_threads))
    end = time.time() - start
    print("Finished downloading %s in %s seconds" % (key, end))

In [8]:
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])


In [9]:
def get_cunks(size_bytes, desired_sections):
    return size_bytes / desired_sections